In [9]:
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.preprocessing import RobustScaler
from time import time

In [2]:
df = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv ' , sep = ';')

In [3]:
features = df.drop('quality', 1).values
def isGood(quality):
    if quality >= 6:
        return 1
    else:
        return 0
df['binary_quality'] = df['quality'].apply(isGood)
label = df['binary_quality'].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=42, stratify=label)

In [5]:
# Now we rescale the feature values
scaler = RobustScaler()
# Fit only to the training data, then apply the transformations to the data
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
def random_grid_search_result(random_search, n_iter_search):
    start = time()
    random_search.fit(X_train, y_train)
    print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
    print(random_search.best_score_)
    print(random_search.best_params_)
    best_clf = random_search.best_estimator_
    best_clf.fit(X_train, y_train)
    predicted = best_clf.predict(X_test)
    print("Accuracy "+ " : "+str(np.mean(predicted == y_test)))

In [12]:
def random_grid_search_knn():
    # Distribution of hyperparameter
    param_dist = {"n_neighbors": list(range(1, 31)),
                  "weights": ['uniform', 'distance'],
                  "algorithm": ['auto', 'ball_tree', 'kd_tree', 'brute'],
                  "leaf_size":list(range(15, 45)),
                  "metric":['euclidean', 'manhattan', 'chebyshev'],
                  "n_jobs": [-1, 1]
                 }
    clf = KNeighborsClassifier()
    n_iter_search = 60
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)
    random_grid_search_result(random_search, n_iter_search)

In [23]:
def random_grid_search_svm():
    # Distribution of hyperparameter
    param_dist = {'C': np.logspace(-3, 2, 6), 
                  'gamma': np.logspace(-3, 2, 6),
                  'kernel': ['linear', 'rbf'],
                  'degree': list(range(1, 10))
                 }
    clf = SVC()
    n_iter_search = 60
    random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)
    random_grid_search_result(random_search, n_iter_search)

In [15]:
random_grid_search_knn()

RandomizedSearchCV took 30.87 seconds for 60 candidates parameter settings.
0.799061767005
{'weights': 'distance', 'n_neighbors': 29, 'n_jobs': -1, 'metric': 'manhattan', 'leaf_size': 41, 'algorithm': 'kd_tree'}
Accuracy  : 0.80625


In [24]:
random_grid_search_svm()

RandomizedSearchCV took 31.18 seconds for 60 candidates parameter settings.
0.758405003909
{'kernel': 'rbf', 'gamma': 0.10000000000000001, 'degree': 5, 'C': 10.0}
Accuracy  : 0.775


In [10]:
dict_classifiers = {
    "Nearest Neighbors": KNeighborsClassifier(n_neighbors=5, weights='distance'),
    "SVC":SVC(),
    "Decision Tree": tree.DecisionTreeClassifier(),
    "Naive Bayes": GaussianNB(),
    "Neural Net": MLPClassifier(alpha = 1)
}

In [11]:
for name, classifier in dict_classifiers.items():
    start_training = time()
    clf = classifier.fit(X_train, y_train)
    end_training = time()
    start_testing = time()
    predicted = clf.predict(X_test)
    end_testing = time()
    print("Accuracy of "+ name + " : "+str(np.mean(predicted == y_test)))
    print("Training time of "+ name + " : "+str(end_training - start_training))
    print("Testing time of "+ name + " : "+str(end_testing - start_testing))

Accuracy of Nearest Neighbors : 0.778125
Training time of Nearest Neighbors : 0.0023598670959472656
Testing time of Nearest Neighbors : 0.009933233261108398
Accuracy of SVC : 0.74375
Training time of SVC : 0.05669999122619629
Testing time of SVC : 0.007154941558837891
Accuracy of Decision Tree : 0.759375
Training time of Decision Tree : 0.007573843002319336
Testing time of Decision Tree : 0.00018978118896484375
Accuracy of Naive Bayes : 0.721875
Training time of Naive Bayes : 0.001920938491821289
Testing time of Naive Bayes : 0.0002658367156982422
Accuracy of Neural Net : 0.75625
Training time of Neural Net : 0.8059592247009277
Testing time of Neural Net : 0.0006508827209472656


In [ ]:
# Histogram
pd.DataFrame.hist(df, figsize = [15,15]);

In [ ]:
# Correlation graph
plt.style.use('ggplot')
correlation_matrix = df.corr()
plt.figure(figsize=(10,8))
ax = sns.heatmap(correlation_matrix, vmax=1, square=True,annot=True,cmap='coolwarm')
plt.show()

In [ ]:
%%time
from sklearn import neighbors, linear_model
knn = neighbors.KNeighborsClassifier(n_neighbors = 3)
knn_model = knn.fit(X_train, y_train)
print('k-NN accuracy for test set: %f' % knn_model.score(X_test, y_test))

In [ ]:
%%time
# SVM
from sklearn import svm
svm_model = svm.SVC()
svm_model.fit(X_train, y_train)
print('SVM accuracy for test set: %f' % svm_model.score(X_test, y_test))

In [ ]:
X_train

In [ ]:
%%time
from sklearn import neighbors, linear_model

knn = neighbors.KNeighborsClassifier(n_neighbors = 3)
knn_model = knn.fit(X_train, y_train)
print('k-NN accuracy for test set after rescaling: %f' % knn_model.score(X_test, y_test))

In [ ]:
from sklearn.metrics import classification_report
y_true, y_pred = y_test, knn_model.predict(X_test)
print(classification_report(y_true, y_pred))

In [ ]:
%%time
# SVM
from sklearn import svm
svm_model = svm.SVC()
svm_model.fit(X_train, y_train)
print('Linear SVM accuracy for test set after rescaling: %f' % svm_model.score(X_test, y_test))

In [ ]:
from sklearn.metrics import classification_report
y_true, y_pred = y_test, svm_model.predict(X_test)
print(classification_report(y_true, y_pred))

In [ ]:
%%time
# Nonlinear SVM
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.svm import LinearSVC

polynomial_svm_model = Pipeline((
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("svm_clf",LinearSVC(C=10, loss="hinge"))))
polynomial_svm_model.fit(X_train, y_train)
print('Nonlinear SVM accuracy for test set: %f' % polynomial_svm_model.score(X_test, y_test))

In [ ]:
from sklearn.metrics import classification_report
y_true, y_pred = y_test, polynomial_svm_model.predict(X_test)
print(classification_report(y_true, y_pred))

In [ ]:
%%time
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

tree_model = DecisionTreeClassifier(max_depth=2)
tree_model.fit(X_train, y_train)
print('Decision Tree accuracy for test set: %f' % tree_model.score(X_test, y_test))

In [ ]:
from sklearn.tree import export_graphviz

export_graphviz(tree_model,
                out_file=None,
                class_names=['0','1'])

In [ ]:
# Plot the relationship between tree depths and prediction power
depth = []
accuracy = []
for x in range(1, 200):
    depth.append(x)
    tree_model = DecisionTreeClassifier(max_depth=x)
    tree_model.fit(X_train, y_train)
    accu = tree_model.score(X_test, y_test)
    accuracy.append(accu)
    plt.xlabel("Depth", fontsize=14)
    plt.ylabel("Accuracy", fontsize=14)
plt.plot(depth, accuracy)
plt.show()

In [ ]:
%%time
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
print('Naive Bayes accuracy for test set: %f' % nb_model.score(X_test, y_test))

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)
print('Naive Bayes accuracy for test set: %f' % nb_model.score(X_test, y_test))

In [ ]:
from sklearn.naive_bayes import BernoulliNB
nb_model = BernoulliNB()
nb_model.fit(X_train, y_train)
print('Naive Bayes accuracy for test set: %f' % nb_model.score(X_test, y_test))

In [ ]:
%%time
# Neural Network
from sklearn.neural_network import MLPClassifier
mlp_model = MLPClassifier(hidden_layer_sizes=(13,13,13),max_iter=500)
mlp_model.fit(X_train, y_train)
print('Neural Network accuracy for test set: %f' % mlp_model.score(X_test, y_test))

In [ ]:
%%time
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix

mlp_model = MLPClassifier(hidden_layer_sizes=(10,10),solver='sgd',learning_rate_init=0.1,max_iter=500, random_state=42)
mlp_model.fit(X_train, y_train)
print('Neural Network accuracy for test set: %f' % mlp_model.score(X_test, y_test))